## Fine-tune Idefics2 for document parsing (PDF -> JSON)

In this notebook, we are going to fine-tune the [Idefics2](https://huggingface.co/docs/transformers/main/en/model_doc/idefics2) model for a document AI use case. Idefics2 is one of the best open-source multimodal models at the time of writing, developed by Hugging Face. Idefics started as a replication of Deepmind's [Flamingo](https://arxiv.org/abs/2204.14198) model, and the second iteration incorporates a lot of advancements in the field such as [NaViT](https://arxiv.org/abs/2307.06304) patching, [Perceiver](https://arxiv.org/abs/2103.03206) resampling and more. It also supports multiple images per training example.

However, explaining how Idefics2 works would desire its own video which I might put on YouTube!

The goal for the model in this notebook is to generate a JSON that contains key fields (like food items and their corresponding prices) from receipts. We will fine-tune Idefics2 on the [CORD](https://huggingface.co/datasets/naver-clova-ix/cord-v2) dataset, which contains (receipt image, ground truth JSON) pairs.

Sources:

* Idefics2 [blog post](https://huggingface.co/blog/idefics2)
* Idefics2 [DocVQA notebook](https://colab.research.google.com/drive/1NtcTgRbSBKN7pYD3Vdx1j9m8pt3fhFDB?usp=sharing#scrollTo=06CMDrH7Kkdy) on which this notebook is based 

Note: this notebook is a direct adaptation of my [original Donut notebook](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Donut/CORD/Fine_tune_Donut_on_a_custom_dataset_(CORD)_with_PyTorch_Lightning.ipynb) for Idefics2. You can view Idefics2 as a more powerful Donut model.

## Load dataset

Let's start by loading the dataset from the hub. Here we use the [CORD](https://huggingface.co/datasets/naver-clova-ix/cord-v2) dataset, created by the [Donut](https://huggingface.co/docs/transformers/en/model_doc/donut) authors (Donut is another powerful - but slightly undertrained document AI model available in the Transformers library). CORD is an important benchmark for receipt understanding. The Donut authors have prepared it in a format that suits vision-language models: we're going to fine-tune it to generate the JSON given the image.

If you want to load your own custom dataset, check out this guide: https://huggingface.co/docs/datasets/image_dataset.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("naver-clova-ix/cord-v2")

Let's check out the dataset:

In [ ]:
dataset

As oftentimes, we get a `DatasetDict` which is a dictionary containing 3 splits, one for training, validation and testing. Each split has 2 features, an image and a corresponding ground truth.

 Let's check the first training example:

In [ ]:
example = dataset['train'][0]
image = example["image"]
# resize image for smaller displaying
width, height = image.size
image = image.resize((int(0.3*width), int(0.3*height)))
image

Let's check the corresponding ground truth:

In [ ]:
example["ground_truth"]

Cool! So this contains a ground truth parsing that we want the model to output given an image. We can read it as json:

In [ ]:
import json

json.loads(example["ground_truth"])

## Load processor

Next, we'll load the processor which is used to prepare the data in the format that the model expects. Neural networks like Idefics2 don't directly take images and text as input, but rather `pixel_values` (which is a resized, rescaled, normalized and optionally splitted version of the receipt images), `input_ids` (which are text token indices in the vocabulary of the model), etc. This is handled by the processor.

### Image splitting

Idefics2's processor has a setting called `do_image_splitting` which can be set to `True`/`False`. This defines how images are prepared for the model, either it will just create 1 image (with `do_image_splitting=False`) or it will create multiple (by splitting the images into multiple patches and also including the original image).

This has an effect on the amount of memory that's going to be used during training: if we use image splitting we'll encounter more memory usage (as several images are created for each receipt image). We'll use the memory friendly version here. Do note that this has an effect on performance; performance is typically higher with the `do_image_splitting=True` setting. The latter is also how the model which we're going to use (https://huggingface.co/HuggingFaceM4/idefics2-8b) was trained.

See also from the [model card](https://huggingface.co/HuggingFaceM4/idefics2-8b):

> do_image_splitting=True is especially needed to boost performance on OCR tasks where a very large image is used as input. For the regular VQA or captioning tasks, this argument can be safely set to False with minimal impact on performance (see the evaluation table above).

### Image resolution

Additionally, one can decrease the maximum image resolution used during training to decrease memory usage. To do so, add `size= {"longest_edge": 448, "shortest_edge": 378}` when initializing the processor (`AutoProcessor.from_pretrained`). In particular, the longest_edge value can be adapted to fit the need (the default value is 980). We recommend using values that are multiples of 14. There are no changes required on the model side.

### Multiple images

Note that the processor supports passing multiple images per training example (this is one of the first in the Transformers library)! This is exciting as it means we could fine-tune Idefics2 on PDFs which contain multiple images. It's not the case for this demo notebook (as each receipt only contains a single image), but I could work on a follow-up notebook in which we fine-tune Idefics2 on a multiple-images PDF dataset (as you'll see, the only difference is providing multiple images to the processor).

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b", do_image_splitting=False)

## Load model

Next, we're going to load the Idefics2 model from the [hub](https://huggingface.co/HuggingFaceM4/idefics2-8b). This is a model with 8 billion trainable parameters. Do note that we load a model here which already has undergone supervised fine-tuning (SFT). The pure pre-trained model (also called "base model") is available here: https://huggingface.co/HuggingFaceM4/idefics2-8b-base. We can benefit from the fine-tuning that the model already has undergone.

Do note that the Idefics2 team is also going to release a chatty version of Idefics2 optimized for chatbot/AI assistant use cases. However in our case we do not care about the chatty aspect, we just want the model to generate perfect JSON given an image of a receipt.

### Full fine-tuning, LoRa and Q-LoRa

As this model has 8 billion trainable parameters, that's going to have quite an impact on the amount of memory used. For reference, fine-tuning a model using the [AdamW optimizer](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html#torch.optim.AdamW) (which is often used to optimize neural networks) with mixed precision, you need about 18 times the amount of parameters in GB of GPU RAM. So in this case, we would need 18x8 billion bytes = 144 GB of GPU RAM if we want to update all the parameters of the model!! That's huge right? And for most people infeasible.

Luckily, some clever people came up with the [LoRa](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora) method (LoRa is short for low-rank adapation). It allows to just freeze the existing weights and only train a couple of adapter layers on top of the base model. Hugging Face offers the separate [PEFT library](https://huggingface.co/docs/peft/main/en/index) for easy use of LoRa, along with other Parameter-Efficient Fine-Tuning methods (that's where the name PEFT comes from).

Moreover, one can not only freeze the existing base model but also quantize it (which means, shrinking down its size). A neural network's parameters are typically saved in either float32 (which means, 32 bits or 4 bytes are used to store each parameter value) or float16 (which means, 16 bits or half a byte - also called half precision). However, with some clever algorithms one can shrink each parameter to just 8 or 4 bits (half a byte!), without significant effect on final performance. Read all about it here: https://huggingface.co/blog/4bit-transformers-bitsandbytes.

This means that we're going to shrink the size of the base Idefics2-8b model considerably using 4-bit quantization, and then only train a couple of adapter layers on top using LoRa (in float16). This idea of combining LoRa with quantization is called Q-LoRa and is the most memory friendly version.

Of course, if you have the memory available, feel free to use full fine-tuning or LoRa without quantization! In case of full fine-tuning, the code snippet below instantiates the model with Flash Attention which considerably speeds up computations.

There exist many forms of quantization, here we leverage the [BitsAndBytes](https://huggingface.co/docs/transformers/main_classes/quantization#transformers.BitsAndBytesConfig) integration.

In [ ]:
from transformers import BitsAndBytesConfig, Idefics2ForConditionalGeneration
from peft import LoraConfig
import torch

USE_LORA = False
USE_QLORA = True

# TODO: so far I get OOM when using PeftModel instead of `add_adapter`
USE_ADD_ADAPTER = True

## Load model

# Three options for training, from the lowest precision training to the highest precision training:
# - QLora
# - Standard Lora
# - Full fine-tuning
if USE_QLORA or USE_LORA:
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
        )
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        quantization_config=bnb_config if USE_QLORA else None,
    )
    if USE_ADD_ADAPTER:
        lora_config = LoraConfig(
            r=8,
            lora_alpha=8,
            lora_dropout=0.1,
            target_modules=".*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$",
            use_dora=False if USE_QLORA else True,
            init_lora_weights="gaussian",
        )
        model.add_adapter(lora_config)
        model.enable_adapters()
else:
    # for full fine-tuning, we can speed up the model using Flash Attention
    # only available on certain devices, see https://github.com/Dao-AILab/flash-attention?tab=readme-ov-file#installation-and-features
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2",
    )

In [ ]:
print(torch.cuda.is_initialized())

## Create PyTorch dataset

Next we'll create a regular [PyTorch dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) which defines the individual items of the dataset. For that, one needs to implement 3 methods: an `init` method, a `len` method (which returns the length of the dataset) and a `getitem` method (which returns items of the dataset).

The `init` method implements 2 things:
* it goes over all the ground truth JSON sequences and turns them into token sequences (which we want the model to generate) using the `json2token` method
* it adds special tokens to the model/tokenizer using the `add_tokens` method for which the model will learn an embedding vector. By doing this, keys which occur in the JSON sequences (like `<s_menu>`, `<s_unitprice>` in our case) will get their own token (and corresponding embedding), whereas otherwise these might have been split up into multiple tokens. Do note that I haven't quantified the performance difference regarding this, not sure it helps a lot but the Donut authors did this so I assume it must benefit training.

Typically, one uses the processor in the `getitem` method to prepare the data in the format that the model expects, but we'll postpone that here for a reason we'll explain later. In our case we're just going to return 2 things: the image and a corresponding ground truth token sequence.

In [ ]:
from torch.utils.data import Dataset
from typing import Any, List, Dict
import random

added_tokens = []

class Idefics2Dataset(Dataset):
    """
    PyTorch Dataset for Idefics2. This class takes a HuggingFace Dataset as input.
    
    Each row, consists of image path(png/jpg/jpeg) and gt data (json/jsonl/txt).
    """

    def __init__(
        self,
        dataset_name_or_path: str,
        split: str = "train",
        sort_json_key: bool = True,
    ):
        super().__init__()

        self.split = split
        self.sort_json_key = sort_json_key

        self.dataset = load_dataset(dataset_name_or_path, split=self.split)
        self.dataset_length = len(self.dataset)

        self.gt_token_sequences = []
        for sample in self.dataset:
            ground_truth = json.loads(sample["ground_truth"])
            if "gt_parses" in ground_truth:  # when multiple ground truths are available, e.g., docvqa
                assert isinstance(ground_truth["gt_parses"], list)
                gt_jsons = ground_truth["gt_parses"]
            else:
                assert "gt_parse" in ground_truth and isinstance(ground_truth["gt_parse"], dict)
                gt_jsons = [ground_truth["gt_parse"]]

            self.gt_token_sequences.append(
                [
                    self.json2token(
                        gt_json,
                        update_special_tokens_for_json_key=self.split == "train",
                        sort_json_key=self.sort_json_key,
                    )
                    for gt_json in gt_jsons  # load json from list of json
                ]
            )

    def json2token(self, obj: Any, update_special_tokens_for_json_key: bool = True, sort_json_key: bool = True):
        """
        Convert an ordered JSON object into a token sequence
        """
        if type(obj) == dict:
            if len(obj) == 1 and "text_sequence" in obj:
                return obj["text_sequence"]
            else:
                output = ""
                if sort_json_key:
                    keys = sorted(obj.keys(), reverse=True)
                else:
                    keys = obj.keys()
                for k in keys:
                    if update_special_tokens_for_json_key:
                        self.add_tokens([fr"<s_{k}>", fr"</s_{k}>"])
                    output += (
                        fr"<s_{k}>"
                        + self.json2token(obj[k], update_special_tokens_for_json_key, sort_json_key)
                        + fr"</s_{k}>"
                    )
                return output
        elif type(obj) == list:
            return r"<sep/>".join(
                [self.json2token(item, update_special_tokens_for_json_key, sort_json_key) for item in obj]
            )
        else:
            obj = str(obj)
            if f"<{obj}/>" in added_tokens:
                obj = f"<{obj}/>"  # for categorical special tokens
            return obj
    
    def add_tokens(self, list_of_tokens: List[str]):
        """
        Add special tokens to tokenizer and resize the token embeddings of the decoder
        """
        newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
        if newly_added_num > 0:
            model.resize_token_embeddings(len(processor.tokenizer))
            added_tokens.extend(list_of_tokens)
    
    def __len__(self) -> int:
        return self.dataset_length

    def __getitem__(self, idx: int) -> Dict:
        """
        Returns one item of the dataset.

        Returns:
            image : the original Receipt image
            target_sequence : tokenized ground truth sequence
        """
        sample = self.dataset[idx]

        # inputs
        image = sample["image"]
        target_sequence = random.choice(self.gt_token_sequences[idx])  # can be more than one, e.g., DocVQA Task 1
        
        return image, target_sequence    

Let's instantiate the PyTorch datasets:

In [ ]:
train_dataset = Idefics2Dataset("naver-clova-ix/cord-v2",  split="train", sort_json_key=False)
val_dataset = Idefics2Dataset("naver-clova-ix/cord-v2", split="validation", sort_json_key=False)

As always, it's important to check your data. Let's check the first example:

In [ ]:
train_example = train_dataset[0]
image, target_sequence = train_example
print(target_sequence)

## Apply PEFT

After resizing the token embedding matrix, we can add our LoRa adapter as an alternative option.

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if not USE_ADD_ADAPTER:
    lora_config = LoraConfig(
            r=8,
            lora_alpha=8,
            lora_dropout=0.1,
            target_modules=".*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$",
            use_dora=False if USE_QLORA else True,
            init_lora_weights="gaussian",
        )

    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)

## Push resized embedding matrices to the hub

Now that we've added special tokens to the model, we'll push them to the hub so that we can use them at inference time.

In [ ]:
print(model.get_input_embeddings().weight.shape)

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# save input embeddings
torch.save(model.get_input_embeddings().weight, "input_embeddings.pt")

api.upload_file(
    path_or_fileobj="input_embeddings.pt",
    path_in_repo="input_embeddings.pt",
    repo_id="nielsr/idefics2-embeddings",
    repo_type="dataset",
)

# save output embeddings
torch.save(model.get_output_embeddings().weight, "output_embeddings.pt")

api.upload_file(
    path_or_fileobj="output_embeddings.pt",
    path_in_repo="output_embeddings.pt",
    repo_id="nielsr/idefics2-embeddings",
    repo_type="dataset",
)

## Define collate functions

Now that we have a PyTorch dataset, we'll define a so-called collator which defines how items of the dataset should be batched together. This is because we typically train neural networks on batches of data (i.e. various images/target sequences combined) rather than one-by-one, using a variant of stochastic-gradient descent or SGD (like Adam, AdamW, etc.).

It's only here that we're going to use the processor to turn the (image, target token sequence) into the format that the model expects (which is `pixel_values`, `input_ids` etc.). The reason we do that here is because it allows for **dynamic padding** of the batches: each batch contains images of various resolutions (as Idefics2 preserves the aspect ratio of images) and ground truth sequences of varying lengths. By only using the processor here, we will pad the `pixel_values` up to the largest one in each batch (rather than padding them all to a fixed resolution upfront), and pad the `input_ids` up to the largest sequence in the batch.

Important here is that we are calling `apply_chat_template`, which applies a so-called chat template which turns the inputs into the format that the model expects. This is VERY important as this is how the model expects inputs to be formatted!! Read all about it here: https://huggingface.co/docs/transformers/main/en/chat_templating. We'll use the text prompt "Extract JSON." which is also going to be used at inference time. This is just a deliberate choice, we could also not include a text prompt and only train the model on the image inputs.

We also decide to limit the length of the text tokens (`input_ids`) to 768 due to memory constraints, feel free to expand if your target token sequences are longer (I'd recommend plotting the average token length of your dataset to determine the optimal value). The Donut authors also used a max length of 768 for CORD.

Labels are created for the model by simply copying the inputs to the LLM (`input_ids`), but with padding tokens replaced by the ignore index of the loss function. This ensures that the model doesn't need to learn to predict padding tokens (used to batch examples together). For Idefics2, this is the `image_token_id` as can be seen [here](https://github.com/huggingface/transformers/blob/6f465d45d98f9eaeef83cfdfe79aecc7193b0f1f/src/transformers/models/idefics2/modeling_idefics2.py#L1860).

Why are the labels a copy of the model inputs, you may ask? The model will internally shift the labels one position to the right so that the model will learn to predict the next token. This can be seen [here](https://github.com/huggingface/transformers/blob/6f465d45d98f9eaeef83cfdfe79aecc7193b0f1f/src/transformers/models/idefics2/modeling_idefics2.py#L1851-L1855).

The collate function for evaluation is different, since there we only need to feed the prompt to the model, as we'll use the `generate()` method to autoregressively generate a completion. Hence we only need to prepare the prompt for the model (notice the `add_generation_prompt=True` flag and the fact that we're not including an "assistant" response). We also pass the ground truth token sequences (`answers`) as this allows us to compute evaluation metrics.

In [ ]:
image_token_id = processor.tokenizer.additional_special_tokens_ids[processor.tokenizer.additional_special_tokens.index("<image>")]
MAX_LENGTH = 768

def train_collate_fn(examples):
    texts = []
    images = []
    for example in examples:
        image, ground_truth = example
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Extract JSON."},
                    {"type": "image"},
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": ground_truth}
                ]
            }
        ]
        text = processor.apply_chat_template(messages, add_generation_prompt=False)
        texts.append(text.strip())
        images.append([image])

    batch = processor(text=texts, images=images, padding=True, truncation=True, max_length=MAX_LENGTH, return_tensors="pt")

    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = image_token_id
    batch["labels"] = labels

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    pixel_values = batch["pixel_values"]
    pixel_attention_mask = batch["pixel_attention_mask"]
    labels = batch["labels"]
    
    return input_ids, attention_mask, pixel_values, pixel_attention_mask, labels


def eval_collate_fn(examples):
    
    # we feed the prompt to the model
    images = []
    texts = []
    answers = []
    for example in examples:
        image, ground_truth = example
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Extract JSON."},
                    {"type": "image"},
                ]
            },
        ]
        text = processor.apply_chat_template(messages, add_generation_prompt=True)
        images.append([image])
        texts.append(text.strip())
        answers.append(ground_truth)

    batch = processor(text=texts, images=images, return_tensors="pt", padding=True) 

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    pixel_values = batch["pixel_values"]
    pixel_attention_mask = batch["pixel_attention_mask"]
    
    return input_ids, attention_mask, pixel_values, pixel_attention_mask, answers

## Define PyTorch LightningModule

There are various ways to train a PyTorch model: one could just use native PyTorch, use the [Trainer API](https://huggingface.co/docs/transformers/en/main_classes/trainer) or frameworks like [Accelerate](https://huggingface.co/docs/accelerate/en/index). In this notebook, I'll use PyTorch Lightning as it allows to easily compute evaluation metrics during training.

Below, we define a [LightningModule](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html), which is the standard way to train a model in PyTorch Lightning. A LightningModule is an `nn.Module` with some additional functionality.

Basically, PyTorch Lightning will take care of all device placements (`.to(device)`) for us, as well as the backward pass, putting the model in training mode, etc.

Notice the difference between a training step and an evaluation step:

- a training step only consists of a forward pass, in which we compute the cross-entropy loss between the model's next token predictions and the ground truth (in parallel for all tokens, this technique is known as "teacher forcing"). The backward pass is handled by PyTorch Lightning.
- an evaluation step consists of making the model autoregressively complete the prompt using the [`generate()`](https://huggingface.co/docs/transformers/v4.40.1/en/main_classes/text_generation#transformers.GenerationMixin.generate) method. After that, we compute an evaluation metric between the predicted sequences and the ground truth ones. This allows us to see how the model is improving over the course of training. The metric we use here is the so-called [Levenhstein edit distance](https://en.wikipedia.org/wiki/Levenshtein_distance). This quantifies how much we would need to edit the predicted token sequence to get the target sequence (the fewer edits the better!). Its optimal value is 0 (which means, no edits need to be made).

Besides that, we define the optimizer to use ([AdamW](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) is a good default choice) and the data loaders, which use the collate functions defined above to batch together items of the PyTorch datasets. Do note that AdamW is a pretty heavy optimizer in terms of memory requirements, but as we're training with QLoRa we only need to store optimizer states for the adapter layers. For full fine-tuning, one could take a look at more memory friendly optimizers such as [8-bit Adam](https://huggingface.co/docs/bitsandbytes/main/en/optimizers).

In [ ]:
import lightning as L
from torch.utils.data import DataLoader
import re
from nltk import edit_distance
import numpy as np


class Idefics2ModelPLModule(L.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

        self.batch_size = config.get("batch_size")

    def training_step(self, batch, batch_idx):

        input_ids, attention_mask, pixel_values, pixel_attention_mask, labels = batch

        outputs = self.model(input_ids=input_ids,
                                attention_mask=attention_mask,
                                pixel_values=pixel_values,
                                pixel_attention_mask=pixel_attention_mask,
                                labels=labels)
        loss = outputs.loss
        
        self.log("train_loss", loss)
        
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        
        input_ids, attention_mask, pixel_values, pixel_attention_mask, answers = batch

        # autoregressively generate token IDs
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                                       pixel_values=pixel_values, pixel_attention_mask=pixel_attention_mask,
                                       max_new_tokens=768)
        # turn them back into text, chopping of the prompt
        # important: we don't skip special tokens here, because we want to see them in the output
        predictions = self.processor.batch_decode(generated_ids[:, input_ids.size(1):], skip_special_tokens=True)

        scores = []
        for pred, answer in zip(predictions, answers):
            pred = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred)
            scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

            if self.config.get("verbose", False) and len(scores) == 1:
                print(f"Prediction: {pred}")
                print(f"    Answer: {answer}")
                print(f" Normed ED: {scores[0]}")

        self.log("val_edit_distance", np.mean(scores))
        
        return scores

    def configure_optimizers(self):
        # you could also add a learning rate scheduler if you want
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.get("lr"))
    
        return optimizer

    def train_dataloader(self):
        return DataLoader(train_dataset, collate_fn=train_collate_fn, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(val_dataset, collate_fn=eval_collate_fn, batch_size=self.batch_size, shuffle=False, num_workers=4)

Let's instantiate it (based on a config dictionary which defines all hyperparameters for training).

The batch size was determined based on PyTorch Lightning's [auto batch size finder](https://lightning.ai/docs/pytorch/stable/advanced/training_tricks.html#batch-size-finder) feature. It tries to find the biggest batch size for your given hardware.

Do note that one can play around with the hyperparameters, I just use good defaults here: 10 epochs, a learning rate of 1e-4 which I found in the original Idefics2 notebook (linked at the top of this notebook), use mixed precision for training (more memory friendly). One could extend this with things like gradient accumulation and gradient checkpointing.

I recommend [this guide](https://huggingface.co/docs/transformers/v4.20.1/en/perf_train_gpu_one) which goes over all tips and tricks regarding maximizing fine-tuning performance on consumer hardware.

In [ ]:
config = {"max_epochs": 10,
          # "val_check_interval": 0.2, # how many times we want to validate during an epoch
          "check_val_every_n_epoch": 1,
          "gradient_clip_val": 1.0,
          "accumulate_grad_batches": 8,
          "lr": 1e-4,
          "batch_size": 2,
          "precision": "16-mixed", # we'll use mixed precision
          # "seed":2022, # can be used for reproducibility
          "warmup_steps": 50,
          "result_path": "./result",
          "verbose": True,
}

model_module = Idefics2ModelPLModule(config, processor, model)

## Define callbacks

Optionally, Lightning allows to define so-called [callbacks](https://lightning.ai/docs/pytorch/stable/extensions/callbacks.html), which are arbitrary pieces of code that can be executed during training.

Here I'm adding a `PushToHubCallback` which will push the model to the [hub](https://huggingface.co/) at the end of every epoch as well as at the end of training. Do note that you could of course also pass the `private=True` flag when pushing to the hub, if you wish to keep your model private. Hugging Face also offers the [Enterprise Hub](https://huggingface.co/enterprise) so that you can easily share models with your colleagues privately in a secure way.

We'll also use the EarlyStopping callback of Lightning, which will automatically stop training once the evaluation metric (edit distance in our case) doesn't improve after 3 epochs.

In [ ]:
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

MODEL_REPO_ID = "nielsr/idefics2-cord-demo-v2"

class PushToHubCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        print(f"Pushing model to the hub, epoch {trainer.current_epoch}")
        pl_module.model.push_to_hub(MODEL_REPO_ID,
                                    commit_message=f"Training in progress, epoch {trainer.current_epoch}")

    def on_train_end(self, trainer, pl_module):
        print(f"Pushing model to the hub after training")
        pl_module.processor.push_to_hub(MODEL_REPO_ID,
                                    commit_message=f"Training done")
        pl_module.model.push_to_hub(MODEL_REPO_ID,
                                    commit_message=f"Training done")

early_stop_callback = EarlyStopping(monitor="val_edit_distance", patience=3, verbose=False, mode="min")

## Train!

Alright, we're set to start training! We will also pass the Weights and Biases logger so that we get see some pretty plots of our loss and evaluation metric during training (do note that you may need to log in the first time you run this, see the [docs](https://docs.wandb.ai/guides/integrations/lightning)).

Do note that this Trainer class supports many more flags! See the docs: https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.trainer.trainer.Trainer.html#lightning.pytorch.trainer.trainer.Trainer.

In [ ]:
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="Idefics2-PL", name="demo-run-cord")

trainer = L.Trainer(
        accelerator="gpu",
        devices=[0],
        max_epochs=config.get("max_epochs"),
        check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
        gradient_clip_val=config.get("gradient_clip_val"),
        accumulate_grad_batches=config.get("accumulate_grad_batches"),
        precision=config.get("precision"),
        num_sanity_val_steps=0,
        logger=wandb_logger,
        callbacks=[PushToHubCallback(), early_stop_callback],
)

trainer.fit(model_module)

## Inference

Let's see if the model has learned something. We'll load the model from the hub first. Notice that, as we only trained adapters on top of the base model, the [repository on the hub](https://huggingface.co/nielsr/idefics2-cord-demo) to which we pushed only contains the weights and configuration of the adapters. This is a very lightweight file smaller than 100 MB.

Thanks to the PEFT integration in Transformers, the `from_pretrained` method will automatically load the weights of the base model ([Idefics2-8B](https://huggingface.co/HuggingFaceM4/idefics2-8b) in our case) as well as the adapter weights.

To reduce inference costs, we'll again load the model in 4 bits by passing a `quantization_config`, in order to reduce memory usage.

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration
import torch

peft_model_id = "nielsr/idefics2-cord-demo"

processor = AutoProcessor.from_pretrained(peft_model_id)

# Define quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)
# Load the base model with adapters on top
model = Idefics2ForConditionalGeneration.from_pretrained(
    peft_model_id,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

Next we need to load the resized token embedding matrices:

In [ ]:
from huggingface_hub import hf_hub_download

filepath = hf_hub_download(repo_id="nielsr/idefics2-embeddings", filename="input_embeddings.pt", repo_type="dataset")
filepath = hf_hub_download(repo_id="nielsr/idefics2-embeddings", filename="output_embeddings.pt", repo_type="dataset")

input_embeddings = torch.load(filepath / "input_embeddings.pt")
output_embeddings = torch.load(filepath / "output_embeddings.pt")

model.resize_token_embeddings(len(processor.tokenizer))
model.set_input_embeddings(input_embeddings)
model.set_output_embeddings(output_embeddings)

We'll take a receipt image of the test set here.

In [ ]:
test_example = dataset["test"][0]
test_image = test_example["image"]
test_image

Next we need to prepare the image for the model, along with the text prompt we used during training. We need to apply the chat template to make sure the format is respected.

Notice that this is exactly the same as what we did for the evaluation data collate function. We need to pass `add_generation_prompt=True` in order to add the prompt with "Assistant:" so that the model will continue it.

In [ ]:
# prepare image and prompt for the model
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Extract JSON."},
            {"type": "image"},
        ]
    },
]
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
print(prompt)

Next we let the model autoregressively generate tokens using the [generate()](https://huggingface.co/docs/transformers/v4.40.1/en/main_classes/text_generation#transformers.GenerationMixin.generate) method, which is recommended for use at inference time. This method feeds each predicted token back into the model as conditioning for each next time step.

Do note that there are various ways of decoding text, here we use greedy decoding which is the default. There are various fancier methods such as beam search and top-k sampling. Refer to [this amazing blog post](https://huggingface.co/blog/how-to-generate) for all details.

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

inputs = processor(text=prompt, images=[test_image], return_tensors="pt").to(DEVICE)

# Generate token IDs
generated_ids = model.generate(**inputs, max_new_tokens=768)

# Decode back into text
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
print(generated_texts)

Based on the Donut model, we could write a `token2json` method which converts the generated token sequence into parsible JSON.

In [ ]:
import re

# let's turn that into JSON
def token2json(tokens, is_inner_value=False, added_vocab=None):
        """
        Convert a (generated) token sequence into an ordered JSON format.
        """
        if added_vocab is None:
            added_vocab = processor.tokenizer.get_added_vocab()

        output = {}

        while tokens:
            start_token = re.search(r"<s_(.*?)>", tokens, re.IGNORECASE)
            if start_token is None:
                break
            key = start_token.group(1)
            key_escaped = re.escape(key)

            end_token = re.search(rf"</s_{key_escaped}>", tokens, re.IGNORECASE)
            start_token = start_token.group()
            if end_token is None:
                tokens = tokens.replace(start_token, "")
            else:
                end_token = end_token.group()
                start_token_escaped = re.escape(start_token)
                end_token_escaped = re.escape(end_token)
                content = re.search(
                    f"{start_token_escaped}(.*?){end_token_escaped}", tokens, re.IGNORECASE | re.DOTALL
                )
                if content is not None:
                    content = content.group(1).strip()
                    if r"<s_" in content and r"</s_" in content:  # non-leaf node
                        value = token2json(content, is_inner_value=True, added_vocab=added_vocab)
                        if value:
                            if len(value) == 1:
                                value = value[0]
                            output[key] = value
                    else:  # leaf nodes
                        output[key] = []
                        for leaf in content.split(r"<sep/>"):
                            leaf = leaf.strip()
                            if leaf in added_vocab and leaf[0] == "<" and leaf[-2:] == "/>":
                                leaf = leaf[1:-2]  # for categorical special tokens
                            output[key].append(leaf)
                        if len(output[key]) == 1:
                            output[key] = output[key][0]

                tokens = tokens[tokens.find(end_token) + len(end_token) :].strip()
                if tokens[:6] == r"<sep/>":  # non-leaf nodes
                    return [output] + token2json(tokens[6:], is_inner_value=True, added_vocab=added_vocab)

        if len(output):
            return [output] if is_inner_value else output
        else:
            return [] if is_inner_value else {"text_sequence": tokens}

Let's print the final JSON!

In [ ]:
generated_json = token2json(generated_texts[0])
print(generated_json)

In [ ]:
for key, value in generated_json.items():
    print(key, value)